In [1]:
%cd d:\Learning\University\UIT\AI\Projects\AI4Finance\AI-Project\src

d:\Learning\University\UIT\AI\Projects\AI4Finance\AI-Project\src


In [25]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Part 1. Install necessary independences

In [3]:
# !pip install wrds
# !pip install swig
# !pip install PyPortfolioOpt
# !pip install pyfolio-reloaded
# !pip install -q condacolab
# !pip install stockstats
# !pip install gym
# !pip install gymnasium
# !pip install stable_baselines3
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig

In [4]:
import sys
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime
from plot import backtest_plot, get_baseline, backtest_stats
%matplotlib inline
from preprocessor.preprocessors import FeatureEngineer, data_split
from env.env_stocktrading import StockTradingEnv
from models.models import DRLAgent,DRLEnsembleAgent
from stable_baselines3.common.logger import configure
from models.algorithms import A2C, PPO, DDPG

from pprint import pprint

import itertools

In [5]:
from config.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    PPO_PARAMS,
    A2C_PARAMS,
    DDPG_PARAMS,
    check_and_make_directories
)
from config.config_tickers import DOW_30_TICKER

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

# Part 2. Read data

We first read the .csv file of our processed data into dataframe.

In [6]:
processed_data = pd.read_csv(DATA_SAVE_DIR + 'processed_full.csv')
processed_data = processed_data.set_index(processed_data.columns[0])
processed_data.index.names = ['']

In [7]:
print(f"Train date: {TRAIN_START_DATE} - {TRAIN_END_DATE}")
print(f"Trade date: {TRADE_START_DATE} - {TRADE_END_DATE}")

Train date: 2015-01-01 - 2020-01-01
Trade date: 2022-01-01 - 2024-06-01


In [8]:
train = data_split(processed_data, TRAIN_START_DATE,TRAIN_END_DATE)
print(len(train))

36482


In [9]:
TRAIN_END_DATE

'2020-01-01'

In [10]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
0,2015-01-02,AAPL,27.847500,27.860001,26.837500,24.402172,212818400.0,4.0,0.0,0.0,-66.666667,100.0,17.790001,0.0
0,2015-01-02,AMGN,160.160004,162.589996,158.600006,122.898552,2605400.0,4.0,0.0,0.0,-66.666667,100.0,17.790001,0.0
0,2015-01-02,AXP,93.169998,93.940002,92.139999,80.772018,2437500.0,4.0,0.0,0.0,-66.666667,100.0,17.790001,0.0
0,2015-01-02,BA,131.070007,131.839996,129.089996,113.657211,4294200.0,4.0,0.0,0.0,-66.666667,100.0,17.790001,0.0
0,2015-01-02,CAT,91.769997,92.370003,90.660004,70.907639,3767900.0,4.0,0.0,0.0,-66.666667,100.0,17.790001,0.0


In [11]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
1257,2019-12-31,UNH,293.660004,294.089996,291.149994,275.170197,2224200.0,1.0,6.764926,68.470048,81.136801,36.764747,13.78,3.600534
1257,2019-12-31,V,187.059998,188.000000,186.539993,182.425278,5273000.0,1.0,2.014772,58.615893,96.599562,8.914116,13.78,3.600534
1257,2019-12-31,VZ,61.209999,61.400002,60.939999,48.101662,8466700.0,1.0,0.256966,56.217642,58.585691,8.088789,13.78,3.600534
1257,2019-12-31,WBA,58.820000,59.470001,58.810001,47.291977,3305100.0,1.0,0.000871,54.048382,-6.217027,16.696635,13.78,3.600534
1257,2019-12-31,WMT,39.833332,39.933334,39.396667,37.065933,14736000.0,1.0,0.045401,51.364065,-65.553194,6.867213,13.78,3.600534


In [12]:
print(f"Train shape: {train.shape}")

Train shape: (36482, 14)


# Part 3. Construct the environment

Calculate and specify the parameters we need for constructing the environment.

In [13]:
INDICATORS = ['macd',
            'rsi_30',
            'cci_30',
            'dx_30']

In [14]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 175


In [15]:
train

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,vix,turbulence
0,2015-01-02,AAPL,27.847500,27.860001,26.837500,24.402172,212818400.0,4.0,0.000000,0.000000,-66.666667,100.000000,17.790001,0.000000
0,2015-01-02,AMGN,160.160004,162.589996,158.600006,122.898552,2605400.0,4.0,0.000000,0.000000,-66.666667,100.000000,17.790001,0.000000
0,2015-01-02,AXP,93.169998,93.940002,92.139999,80.772018,2437500.0,4.0,0.000000,0.000000,-66.666667,100.000000,17.790001,0.000000
0,2015-01-02,BA,131.070007,131.839996,129.089996,113.657211,4294200.0,4.0,0.000000,0.000000,-66.666667,100.000000,17.790001,0.000000
0,2015-01-02,CAT,91.769997,92.370003,90.660004,70.907639,3767900.0,4.0,0.000000,0.000000,-66.666667,100.000000,17.790001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,2019-12-31,UNH,293.660004,294.089996,291.149994,275.170197,2224200.0,1.0,6.764926,68.470048,81.136801,36.764747,13.780000,3.600534
1257,2019-12-31,V,187.059998,188.000000,186.539993,182.425278,5273000.0,1.0,2.014772,58.615893,96.599562,8.914116,13.780000,3.600534
1257,2019-12-31,VZ,61.209999,61.400002,60.939999,48.101662,8466700.0,1.0,0.256966,56.217642,58.585691,8.088789,13.780000,3.600534
1257,2019-12-31,WBA,58.820000,59.470001,58.810001,47.291977,3305100.0,1.0,0.000871,54.048382,-6.217027,16.696635,13.780000,3.600534


In [16]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [17]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Part 4. Train DRL Agents

In [19]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_ensemble = False

### Agent 1: A2C

In [43]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c", model_kwargs= A2C_PARAMS)

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + 'a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [44]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=20000) if if_using_a2c else None

--------------------------------------
| time/                 |            |
|    fps                | 88         |
|    iterations         | 100        |
|    time_elapsed       | 5          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -41.2      |
|    explained_variance | -3.73      |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -2.04      |
|    reward             | 0.06526817 |
|    std                | 1          |
|    value_loss         | 0.375      |
--------------------------------------


KeyboardInterrupt: 

In [ ]:
trained_a2c.save(TRAINED_MODEL_DIR + "agent_a2c") if if_using_a2c else None

### Agent 2: DDPG

In [20]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg", model_kwargs= DDPG_PARAMS)

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + 'ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to results/ddpg


In [21]:
trained_ddpg = agent.train_model(model=model_ddpg,
                            tb_log_name='ddpg',
                            total_timesteps=20000) if if_using_ddpg else None

KeyboardInterrupt: 

In [ ]:
trained_ddpg.save(TRAINED_MODEL_DIR + "agent_ddpg") if if_using_ddpg else None

### Agent 3: PPO

In [23]:
agent = DRLAgent(env = env_train)

model_ppo = agent.get_model("ppo", model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + 'ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to results/ppo


In [24]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=20000) if if_using_ppo else None

KeyboardInterrupt: 

In [ ]:
trained_ppo.save(TRAINED_MODEL_DIR + "agent_ppo") if if_using_ppo else None

## Ensemble Strategy

In [ ]:
ensemble_data = processed_data

In [ ]:
rebalance_window = 63
validation_window = 63

env_ensemble_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5
}

ensemble_agent = DRLEnsembleAgent(df=ensemble_data,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TRADE_START_DATE,TRADE_END_DATE),
                 rebalance_window=rebalance_window,
                 validation_window=validation_window,
                 **env_ensemble_kwargs) if if_using_ensemble else None

In [ ]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

timesteps_dict = {'a2c' : 20000,
                 'ppo' : 20000,
                 'ddpg' : 20000,
                 }

In [ ]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 timesteps_dict) if if_using_ensemble else None

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
======a2c Validation from:  2022-07-06 to  2022-10-04
Episode: 1
a2c Sharpe Ratio:  -0.26117398891072374
======ddpg Training========
{'buffer_size': 10000, 'learning_rate': 0.0005, 'batch_size': 64}
Using cuda device
Logging to tensorboard_log//ddpg/ddpg_252_1
Episode: 12
Episode: 13
Episode: 14
Episode: 15
day: 1888, episode: 15
begin_total_asset: 1000000.00
end_total_asset: 2426844.46
total_reward: 1426844.46
total_cost: 1651.84
total_trades: 18948
Sharpe: 0.699
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 59         |
|    time_elapsed    | 126        |
|    total_timesteps | 7556       |
| train/             |            |
|    actor_loss      | -0.668     |
|    critic_loss     | 49.3       |
|    learning_rate   | 0.0005     |
|    n_updates       | 7455       |
|    reward          | -3.2919238 |
-----------------------------------
Episod

In [ ]:
df_summary if if_using_ensemble else None

,Iter,Val Start,Val End,Model Used,A2C Sharpe,PPO Sharpe,DDPG Sharpe
0,126,2022-01-03,2022-04-04,DDPG,-0.141601,-0.193884,-0.019706
1,189,2022-04-04,2022-07-06,A2C,-0.133007,-0.226792,-0.226485
2,252,2022-07-06,2022-10-04,DDPG,-0.261174,-0.32862,-0.022517
3,315,2022-10-04,2023-01-04,A2C,0.299137,0.255318,0.225312
4,378,2023-01-04,2023-04-05,DDPG,0.034476,-0.117529,0.041095
5,441,2023-04-05,2023-07-07,DDPG,-0.066499,-0.046204,0.029138
6,504,2023-07-07,2023-10-05,DDPG,-0.34853,-0.230318,0.013958
7,567,2023-10-05,2024-01-05,DDPG,0.524582,0.524218,0.584759
